# <span style = 'color: #81A5FF'>  Exploratory Data Analysis and Cleaning </span>

### 1. Importing Libraries and data

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#from currency_converter import CurrencyConverter
import datetime
import requests
# from geopy.geocoders import Nominatim
from functions import *
from locations import *


%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'currency_converter'

In [5]:
data = pd.read_csv('data/alltheforkscrapes2.csv', index_col=0)

/var/folders/m6/p24b6kxx6r1f28pjz4rlyh6h0000gn/T/ipykernel_2971/2888128206.py:1: DtypeWarning: Columns (292,300,301,302,303,304,305,306,307) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/alltheforkscrapes2.csv', index_col=0)


In [6]:
data.reset_index(inplace=True)
data.drop(['index'], axis=1, inplace=True)

In [7]:
data

,address,averagePrice,chefName,cuisine,currency,customerPhotos/0,customerPhotos/1,customerPhotos/2,customerPhotos/3,customerPhotos/4,...,photos/96,photos/97,photos/98,photos/99,photos/100,photos/101,photos/102,photos/103,photos/104,photos/105
0,"R. José Fontana 10,8200-295,Albufeira",14,NaN,International food,EUR,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"R. Almeida Garrett 71,8200-272,Albufeira",15,NaN,International food,EUR,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"R. da Torre Velha,8200,Albufeira",24,Yana Malayko,Japanese,EUR,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Tv. Antero de Quental 4B r/c,8200-296,Albufeira",14,Iqbal Hossain Mohon,Indian,EUR,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"R. Alexandre Herculano 11,8200-269,Albufeira",15,Romilson Santos,Portuguese,EUR,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2552,"Largo da Cruz Velha,4760-107,Vila Nova de Fama...",15,NaN,Portuguese,EUR,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2553,"Alameda Cónego Joaquim Fernandes 160,4760-884,...",18,Gatto Ginno,Italian,EUR,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2554,"R. Augusto Correia 42,4760-101,Vila Nova de Fa...",16,NaN,Portuguese,EUR,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2555,"Av. 25 de Abril 22 4760,4760-101,Vila Nova de ...",25,Miguel Frias,Steakhouse,EUR,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
data[data.duplicated()]

,address,averagePrice,chefName,cuisine,currency,customerPhotos/0,customerPhotos/1,customerPhotos/2,customerPhotos/3,customerPhotos/4,...,photos/96,photos/97,photos/98,photos/99,photos/100,photos/101,photos/102,photos/103,photos/104,photos/105
401,"142 Chiswick High Road,W4 1PU,London",81,Nathan Cornwell,British,GBP,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402,"34 Rue Beaurepaire,75010,Paris",45,Masahiro MORIYA,French,EUR,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
403,"17 Rue Bergère,75009,Paris",55,Satoshi HORIUCHI,French,EUR,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
404,"1 Rue Ambroise Thomas,75009,Paris",80,Florian Barbarot,French,EUR,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
405,"Carrer de Bala Roja, 1,07001,Palma de Mallorca",150,FERNANDO P. ARELLANO,Mediterranean,EUR,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2184,"8200-278,8200-278,Albufeira",25,NaN,Mediterranean,EUR,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2185,"R. Torre da Medronheira 1,8200-650,Olhos de Água",15,Tati Justino,Mediterranean,EUR,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2186,"Urbanizaçao Vila Praia, Rua Torre da Medronhei...",37,Daniel Holdsworth Delgado,European,EUR,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,https://res.cloudinary.com/tf-lab/image/upload...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2187,"EPIC SANA Algarve Hotel Pinhal do Concelho, Pr...",33,Luis Mourao,Portuguese,EUR,https://res.cloudinary.com/tf-lab/image/upload...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#Drop duplicated rows
data.drop_duplicates(inplace=True, keep='last')

In [10]:
data.reset_index(inplace=True)
data.drop(['index'], axis=1, inplace=True)

### 2. Summary Statistics

In [11]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
averagePrice,1792.0,2.702734e+01,2.429775e+01,9.000000e+00,1.500000e+01,2.000000e+01,2.800000e+01,3.500000e+02
latitude,1792.0,4.038920e+01,4.289454e+00,3.701935e+01,3.867652e+01,3.869686e+01,4.115794e+01,5.340000e+01
longitude,1792.0,-8.151668e+00,1.686299e+00,-9.381659e+00,-9.165105e+00,-8.629105e+00,-8.147684e+00,-2.983333e+00
maxPartySize,1547.0,5.167033e+01,2.674151e+01,4.000000e+00,4.000000e+01,6.000000e+01,6.000000e+01,3.700000e+02
phone,1078.0,3.348657e+11,7.634754e+10,3.314014e+10,3.512189e+11,3.512890e+11,3.519204e+11,4.478807e+11
...,...,...,...,...,...,...,...,...
reviewList/18/serviceRatingValue,1075.0,9.374884e+00,1.440283e+00,2.000000e+00,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01
reviewList/19/ambienceRatingValue,1053.0,9.137702e+00,1.513260e+00,2.000000e+00,8.000000e+00,1.000000e+01,1.000000e+01,1.000000e+01
reviewList/19/foodRatingValue,1053.0,9.327635e+00,1.478607e+00,2.000000e+00,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01
reviewList/19/ratingValue,1053.0,9.291073e+00,1.258068e+00,2.000000e+00,9.000000e+00,1.000000e+01,1.000000e+01,1.000000e+01


In [12]:
data.describe(include='object').T

,count,unique,top,freq
address,1792,1618,"Rua Alexandre O'Neill,8200-343,Albufeira",6
chefName,741,611,Carlos Guerreiro,6
cuisine,1789,56,Portuguese,532
currency,1792,5,EUR,1747
customerPhotos/0,1685,1551,https://res.cloudinary.com/tf-lab/image/upload...,3
...,...,...,...,...
photos/101,1,1,https://res.cloudinary.com/tf-lab/image/upload...,1
photos/102,1,1,https://res.cloudinary.com/tf-lab/image/upload...,1
photos/103,1,1,https://res.cloudinary.com/tf-lab/image/upload...,1
photos/104,1,1,https://res.cloudinary.com/tf-lab/image/upload...,1


### 3. Handling Locations

In [13]:
locations = data[['name','address', 'location', 'longitude', 'latitude']].copy()

In [14]:
#Finds last occurence of the address to determine the city
locations['location2'] = locations['address'].map(lambda x: str(x).split(',')[-1])

#Finds the first occurence within "location" to find city -> this was how we eextracted the data
locations['location3'] = locations['location'].map(lambda x: str(x).split(',')[0])

In [15]:
#Finding indexes from observations not based in Portugal and removing them from the original data and creating a new dataframe with only portuguese location info
foreign_indexes = list((locations[(locations['location'].isna())]).index)
data.drop(foreign_indexes, inplace=True, axis=0)
data.reset_index(inplace=True)
data.drop(['index'], axis=1, inplace=True)
pt_locations = locations[locations['location'].isna() == False].copy()
pt_locations.reset_index(inplace=True)
pt_locations.drop(['index'], axis=1, inplace=True)

In [16]:
#Pre processing address to further find latitude and longitude coordinates
pt_locations['address'] = pt_locations['address'].apply(preprocess_address)

NameError: name 'preprocess_address' is not defined

In [17]:
# pt_locations[['latitude', 'longitude']] = pt_locations['address'].apply(lambda x: pd.Series(find_coordinates2(x)))

In [18]:
# pt_locations.to_csv('portuguese_locations.csv')

In [19]:
#Step intermédio enquanto o notebook nao estiver finalizado
pt_locations2 = pd.read_csv('data/portuguese_locations.csv', index_col=0)
pt_locations[['latitude', 'longitude']] = pt_locations2[['latitude', 'longitude']].copy()

In [20]:
data[['address', 'latitude', 'longitude', 'location', 'city']] = pt_locations[['address', 'latitude', 'longitude', 'location2', 'location3']].copy()

### 4. Handling Missing Values

In [21]:
null_columns = data.isnull().any()
null_columns = null_columns[null_columns].index
list(null_columns)

['chefName',
 'customerPhotos/0',
 'customerPhotos/1',
 'customerPhotos/2',
 'customerPhotos/3',
 'customerPhotos/4',
 'customerPhotos/5',
 'customerPhotos/6',
 'customerPhotos/7',
 'customerPhotos/8',
 'customerPhotos/9',
 'customerPhotos/10',
 'customerPhotos/11',
 'customerPhotos/12',
 'customerPhotos/13',
 'customerPhotos/14',
 'customerPhotos/15',
 'customerPhotos/16',
 'customerPhotos/17',
 'customerPhotos/18',
 'customerPhotos/19',
 'description',
 'maxPartySize',
 'offer',
 'openingHours',
 'paymentAccepted/0',
 'paymentAccepted/1',
 'paymentAccepted/2',
 'paymentAccepted/3',
 'paymentAccepted/4',
 'paymentAccepted/5',
 'paymentAccepted/6',
 'paymentAccepted/7',
 'paymentAccepted/8',
 'phone',
 'photos/1',
 'photos/2',
 'photos/3',
 'photos/4',
 'photos/5',
 'photos/6',
 'photos/7',
 'photos/8',
 'photos/9',
 'photos/10',
 'photos/11',
 'photos/12',
 'photos/13',
 'photos/14',
 'photos/15',
 'photos/16',
 'photos/17',
 'photos/18',
 'photos/19',
 'photos/20',
 'photos/21',
 'ph

#### 4.1. Simple Imputations - Offer, Chef Name, Phone Number

In [22]:
data['offer'] = np.where(data['offer'].isnull(), 'No Offer', data['offer'])

In [23]:
data['chefName'] = np.where(data['chefName'].isnull(), 'Not Applicable', data['chefName'])

In [24]:
data['phone'] = data['phone'].astype(str)
data['phone'] = data['phone'].apply(lambda x: x[3:12] if x != 'nan' else 'Not Available')
data['phone'] = np.where(data['phone'].str.endswith('.'), '966287253', data['phone'])

In [25]:
data['openingHours'].fillna('Not Available', inplace=True)

Aqui falta fazer description, ratingValue, maxPartySize, reviewCount

### 5. Dealing with Schedule Format

In [26]:
#Clearing the openingHours column
data['schedule'] = data['openingHours'].apply(lambda x: clean_openinghours(x))

NameError: name 'clean_openinghours' is not defined

### 6. Exploratory Data Analysis (through Sections)

#### 6.1. Photos

In [27]:
photos = ['photo', 'customerPhotos/0','customerPhotos/1', 'customerPhotos/2', 'customerPhotos/3', 'customerPhotos/4', 'customerPhotos/5', 'customerPhotos/6',
 'customerPhotos/7', 'customerPhotos/8', 'customerPhotos/9', 'customerPhotos/10', 'customerPhotos/11', 'customerPhotos/12', 'customerPhotos/13',
 'customerPhotos/14', 'customerPhotos/15', 'customerPhotos/16', 'customerPhotos/17', 'customerPhotos/18', 'customerPhotos/19', 'photos/1',
 'photos/2', 'photos/3', 'photos/4', 'photos/5', 'photos/6', 'photos/7','photos/8','photos/9','photos/10', 'photos/11', 'photos/12', 'photos/13', 'photos/14',
 'photos/15','photos/16', 'photos/17', 'photos/18', 'photos/19', 'photos/20', 'photos/21', 'photos/22', 'photos/23', 'photos/24', 'photos/25', 'photos/26', 'photos/27',
 'photos/28', 'photos/29', 'photos/30', 'photos/31', 'photos/32', 'photos/33', 'photos/34', 'photos/35', 'photos/36', 'photos/37', 'photos/38', 'photos/39', 'photos/40',
 'photos/41', 'photos/42', 'photos/43', 'photos/44', 'photos/45', 'photos/46', 'photos/47', 'photos/48', 'photos/49', 'photos/50', 'photos/51', 'photos/52','photos/53',
 'photos/54', 'photos/55', 'photos/56', 'photos/57', 'photos/58', 'photos/59', 'photos/60', 'photos/61', 'photos/62', 'photos/63', 'photos/64', 'photos/65', 'photos/66',
 'photos/67','photos/68','photos/69', 'photos/70', 'photos/71', 'photos/72', 'photos/73', 'photos/74', 'photos/75', 'photos/76', 'photos/77', 'photos/78', 'photos/79',
 'photos/80', 'photos/81', 'photos/82', 'photos/83', 'photos/84', 'photos/85', 'photos/86', 'photos/87', 'photos/88', 'photos/89', 'photos/90',  'photos/91', 'photos/92',
 'photos/93', 'photos/94', 'photos/95', 'photos/96', 'photos/97', 'photos/98', 'photos/99', 'photos/100', 'photos/101', 'photos/102', 'photos/103', 'photos/104', 'photos/105']

In [28]:
df_photos = data[photos].copy()

In [29]:
df_photos['photos/1'].unique()

array(['https://res.cloudinary.com/tf-lab/image/upload/f_auto,q_auto,w_664,h_374/restaurant/0fb87f67-79d8-4ad5-bb06-6e9d668400cd/8d85bcd4-f6d9-47a3-967d-6be6f376d251.jpg',
       'https://res.cloudinary.com/tf-lab/image/upload/f_auto,q_auto,w_664,h_374/restaurant/cf797b76-9f49-4e2d-bd25-89b05d312ff3/5ffd061d-3b83-43d9-aa13-7bf39afecfd7.jpg',
       'https://res.cloudinary.com/tf-lab/image/upload/f_auto,q_auto,w_664,h_374/restaurant/25442534-4b23-40e5-91a5-2eb2da7b7a92/86b9de08-b5f8-4c6f-97c9-f8ee4dd0ad4f.jpg',
       ...,
       'https://res.cloudinary.com/tf-lab/image/upload/f_auto,q_auto,w_664,h_374/restaurant/7252fa4a-63a4-43bb-b4d5-83e3c48db045/63238949-2ad2-47e7-a812-8c4a215fbef9.jpg',
       'https://res.cloudinary.com/tf-lab/image/upload/f_auto,q_auto,w_664,h_374/restaurant/97249dc8-13a3-4473-bd9b-c9de1ada5f54/8bbb15f6-03df-4f59-8daa-73643c0b737d.jpg',
       'https://res.cloudinary.com/tf-lab/image/upload/f_auto,q_auto,w_664,h_374/restaurant/ac8c8d50-6309-4eae-bf6a-bb59fa7b9646

#### 6.2. Reviews

In [30]:
reviews = ['reviewList/0/ambienceRatingValue','reviewList/0/date', 'reviewList/0/foodRatingValue', 'reviewList/0/ratingValue', 'reviewList/0/review',
 'reviewList/0/reviewerName', 'reviewList/0/serviceRatingValue', 'reviewList/1/ambienceRatingValue', 'reviewList/1/date', 'reviewList/1/foodRatingValue', 'reviewList/1/ratingValue',
 'reviewList/1/review', 'reviewList/1/reviewerName', 'reviewList/1/serviceRatingValue', 'reviewList/2/ambienceRatingValue', 'reviewList/2/date', 'reviewList/2/foodRatingValue',
 'reviewList/2/ratingValue', 'reviewList/2/review', 'reviewList/2/reviewerName', 'reviewList/2/serviceRatingValue', 'reviewList/3/ambienceRatingValue','reviewList/3/date',
 'reviewList/3/foodRatingValue', 'reviewList/3/ratingValue', 'reviewList/3/review', 'reviewList/3/reviewerName', 'reviewList/3/serviceRatingValue', 'reviewList/4/ambienceRatingValue',
 'reviewList/4/date','reviewList/4/foodRatingValue', 'reviewList/4/ratingValue', 'reviewList/4/review', 'reviewList/4/reviewerName', 'reviewList/4/serviceRatingValue',
 'reviewList/5/ambienceRatingValue', 'reviewList/5/date', 'reviewList/5/foodRatingValue', 'reviewList/5/ratingValue', 'reviewList/5/review', 'reviewList/5/reviewerName',
 'reviewList/5/serviceRatingValue', 'reviewList/6/ambienceRatingValue', 'reviewList/6/date', 'reviewList/6/foodRatingValue', 'reviewList/6/ratingValue', 'reviewList/6/review',
 'reviewList/6/reviewerName', 'reviewList/6/serviceRatingValue', 'reviewList/7/ambienceRatingValue', 'reviewList/7/date', 'reviewList/7/foodRatingValue', 'reviewList/7/ratingValue',
 'reviewList/7/review', 'reviewList/7/reviewerName', 'reviewList/7/serviceRatingValue', 'reviewList/8/ambienceRatingValue', 'reviewList/8/date', 'reviewList/8/foodRatingValue',
 'reviewList/8/ratingValue', 'reviewList/8/review', 'reviewList/8/reviewerName', 'reviewList/8/serviceRatingValue', 'reviewList/9/ambienceRatingValue', 'reviewList/9/date',
 'reviewList/9/foodRatingValue', 'reviewList/9/ratingValue', 'reviewList/9/review', 'reviewList/9/reviewerName', 'reviewList/9/serviceRatingValue', 'reviewList/10/ambienceRatingValue',
 'reviewList/10/date', 'reviewList/10/foodRatingValue', 'reviewList/10/ratingValue', 'reviewList/10/review', 'reviewList/10/reviewerName', 'reviewList/10/serviceRatingValue',
 'reviewList/11/ambienceRatingValue', 'reviewList/11/date', 'reviewList/11/foodRatingValue', 'reviewList/11/ratingValue', 'reviewList/11/review', 'reviewList/11/reviewerName',
 'reviewList/11/serviceRatingValue', 'reviewList/12/ambienceRatingValue', 'reviewList/12/date', 'reviewList/12/foodRatingValue', 'reviewList/12/ratingValue', 'reviewList/12/review',
 'reviewList/12/reviewerName', 'reviewList/12/serviceRatingValue', 'reviewList/13/ambienceRatingValue', 'reviewList/13/date', 'reviewList/13/foodRatingValue', 'reviewList/13/ratingValue',
 'reviewList/13/review', 'reviewList/13/reviewerName', 'reviewList/13/serviceRatingValue', 'reviewList/14/ambienceRatingValue', 'reviewList/14/date',
 'reviewList/14/foodRatingValue', 'reviewList/14/ratingValue', 'reviewList/14/review', 'reviewList/14/reviewerName', 'reviewList/14/serviceRatingValue', 'reviewList/15/ambienceRatingValue',
 'reviewList/15/date', 'reviewList/15/foodRatingValue', 'reviewList/15/ratingValue', 'reviewList/15/review', 'reviewList/15/reviewerName', 'reviewList/15/serviceRatingValue',
 'reviewList/16/ambienceRatingValue', 'reviewList/16/date', 'reviewList/16/foodRatingValue', 'reviewList/16/ratingValue', 'reviewList/16/review', 'reviewList/16/reviewerName',
 'reviewList/16/serviceRatingValue', 'reviewList/17/ambienceRatingValue', 'reviewList/17/date', 'reviewList/17/foodRatingValue', 'reviewList/17/ratingValue', 'reviewList/17/review',
 'reviewList/17/reviewerName', 'reviewList/17/serviceRatingValue', 'reviewList/18/ambienceRatingValue', 'reviewList/18/date', 'reviewList/18/foodRatingValue', 'reviewList/18/ratingValue',
 'reviewList/18/review', 'reviewList/18/reviewerName', 'reviewList/18/serviceRatingValue', 'reviewList/19/ambienceRatingValue', 'reviewList/19/date', 'reviewList/19/foodRatingValue',
 'reviewList/19/ratingValue', 'reviewList/19/review', 'reviewList/19/reviewerName', 'reviewList/19/serviceRatingValue']

In [31]:
df_reviews = data[reviews].copy()

In [32]:
ambience = []
food = []
service = []
for col in df_reviews.columns:
    if 'ambienceRatingValue' in col:
        ambience.append(col)
    elif 'foodRatingValue' in col:
        food.append(col)
    elif 'serviceRatingValue' in col:
        service.append(col)

In [33]:
#Creating three new columns regarding ratings per category using the mean values of the user-based ratings
df_reviews['ambienceRatingSummary'] = df_reviews[ambience].mean(axis=1)
df_reviews['foodRatingSummary'] = df_reviews[food].mean(axis=1)
df_reviews['serviceRatingSummary'] = df_reviews[service].mean(axis=1)

In [34]:
data[['ambienceRatingSummary', 'foodRatingSummary', 'serviceRatingSummary']] = df_reviews[['ambienceRatingSummary', 'foodRatingSummary', 'serviceRatingSummary']].copy()

#### 6.3. Payment Methods

In [35]:
payments = ['paymentAccepted/0','paymentAccepted/1', 'paymentAccepted/2', 'paymentAccepted/3', 'paymentAccepted/4',
 'paymentAccepted/5', 'paymentAccepted/6', 'paymentAccepted/7', 'paymentAccepted/8', 'paymentAccepted/9', 'paymentAccepted/10']

In [36]:
df_payments = data[payments].copy()

In [37]:
df_payments.fillna(0, inplace=True)

In [38]:
df_payments['paymentAcceptedSummary'] = df_payments.apply(lambda row: [row[col] for col in df_payments.columns if row[col] != 0], axis=1)

In [39]:
data['paymentAcceptedSummary'] = df_payments['paymentAcceptedSummary'].copy()

In [40]:

data_exploded_pay = data['paymentAcceptedSummary'].explode()
payment_counts = data_exploded_pay.value_counts()

print(payment_counts)

paymentAcceptedSummary
Credit Card          1345
Visa                 1197
Mastercard           1049
Visa Electron         560
Maestro Card          274
Cabal Debit Card      250
Cabal Credit Card     248
American Express      212
MobilePay             157
Apple Pay              60
Cash Only              39
Diners Club Card       24
Voucher                21
Check                   8
Bancontact              5
Elo                     3
Clave Debit Card        2
Sodexo                  2
Rede Shop               1
EC card                 1
Clave Credit Card       1
Name: count, dtype: int64


In [41]:
data.iloc[92]

address                   Aveiro Business Center Rua da Igreja, nº79 Nª ...
averagePrice                                                             12
chefName                                                     Not Applicable
cuisine                                                       Mediterranean
currency                                                                EUR
                                                ...                        
city                                                                 Aveiro
ambienceRatingSummary                                              9.571429
foodRatingSummary                                                  9.285714
serviceRatingSummary                                               9.571429
paymentAcceptedSummary    [American Express, Credit Card, Mastercard, Visa]
Name: 92, Length: 312, dtype: object

In [42]:
#Replace 'Cash Only' with 'Cash'
data['paymentAcceptedSummary'] = [x if 'Cash Only' not in x else [item.replace('Cash Only', 'Cash') for item in x] for x in data['paymentAcceptedSummary']]
data['paymentAcceptedSummary'].iloc[31]

['Credit Card', 'Mastercard', 'Visa', 'Visa Electron']

#### 6.4. Tags

In [102]:
df_tags = data[['tags/0','tags/1', 'tags/2', 'tags/3', 'tags/4', 'tags/5', 'tags/6', 'tags/7',]].copy()
df_tags

,tags/0,tags/1,tags/2,tags/3,tags/4,tags/5,tags/6,tags/7
0,YUMS x2,International food,NaN,NaN,NaN,NaN,NaN,NaN
1,International food,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Japanese,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Indian,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Portuguese,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1629,Portuguese,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1630,Italian,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1631,Portuguese,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1632,INSIDER,Steakhouse,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
df_tags['tags/0'].unique()

array(['YUMS x2', 'International food', 'Japanese', 'Indian',
       'Portuguese', 'Pizzeria', 'MICHELIN', 'International',
       'Mediterranean', 'Fusion', 'INSIDER', 'Nepalese', 'Italian',
       'European', 'Seafood', 'Vegan cuisine', 'Traditional cuisine',
       'Steakhouse', 'Regional', 'Greek', 'Vegetarian', 'Varied',
       'Grilled', 'Thai', 'Mexican', 'Local', 'French', 'Pub grub',
       'Brazilian', 'Venezuelan', 'Peruvian', 'Korean', 'American',
       'Meat Cuisine', 'Asian', 'Toscano', 'African', 'Syrian', 'Iranian',
       'Lebanese', 'Tibetan', 'Vietnamese', 'Argentinian', 'Pugliese',
       'Spanish'], dtype=object)

In [104]:
df_tags['tags/1'].unique()

array(['International food', nan, 'Italian', 'INSIDER', 'Japanese',
       'Portuguese', 'Accepting my yums', 'European',
       'Traditional cuisine', 'Welcome', 'Varied', 'International',
       'Asian', 'Mediterranean', 'Thai', 'Steakhouse', 'South American',
       'Fusion', 'Pizzeria', 'MICHELIN', 'Meat Cuisine', 'Brazilian',
       'Seafood', 'Regional', 'Grilled', 'Mexican', 'Spanish', 'Indian',
       'Peruvian', 'Cantonese', 'Vegan cuisine', 'Local', 'French',
       'Chinese'], dtype=object)

In [105]:
df_tags['tags/2'].unique()

array([nan, 'Portuguese', 'International food', 'Indian', 'European',
       'Italian', 'Accepting my yums', 'Mediterranean', 'Japanese',
       'International', 'Vegetarian', 'Traditional cuisine'], dtype=object)

In [106]:
df_tags['tags/3'].unique()

array([nan, 'Accepting my yums'], dtype=object)

In [107]:
df_tags['tags/4'].unique()

array([nan], dtype=object)

In [108]:
df_tags['tags/5'].unique()

array([nan], dtype=object)

In [109]:
df_tags['tags/6'].unique()

array([nan], dtype=object)

In [110]:
df_tags['tags/7'].unique()

array([nan], dtype=object)

In [111]:
df_tags.to_csv('tags.csv', index = True)

In [112]:
df_tags['MICHELIN'] = df_tags.apply(lambda row: 1 if 'MICHELIN' in row.values else 0, axis=1)

In [113]:
df_tags[df_tags['MICHELIN']==1]

,tags/0,tags/1,tags/2,tags/3,tags/4,tags/5,tags/6,tags/7,MICHELIN
9,MICHELIN,INSIDER,Portuguese,NaN,NaN,NaN,NaN,NaN,1
101,MICHELIN,INSIDER,International food,Accepting my yums,NaN,NaN,NaN,NaN,1
130,MICHELIN,INSIDER,International food,NaN,NaN,NaN,NaN,NaN,1
157,MICHELIN,INSIDER,European,NaN,NaN,NaN,NaN,NaN,1
174,MICHELIN,INSIDER,Portuguese,NaN,NaN,NaN,NaN,NaN,1
246,MICHELIN,INSIDER,Portuguese,NaN,NaN,NaN,NaN,NaN,1
249,MICHELIN,INSIDER,European,NaN,NaN,NaN,NaN,NaN,1
262,MICHELIN,INSIDER,Italian,NaN,NaN,NaN,NaN,NaN,1
269,MICHELIN,Portuguese,NaN,NaN,NaN,NaN,NaN,NaN,1
286,MICHELIN,INSIDER,European,NaN,NaN,NaN,NaN,NaN,1


In [114]:
df_tags = df_tags.replace('MICHELIN', np.nan)

In [115]:
df_tags[df_tags['MICHELIN']==1]

,tags/0,tags/1,tags/2,tags/3,tags/4,tags/5,tags/6,tags/7,MICHELIN
9,NaN,INSIDER,Portuguese,NaN,NaN,NaN,NaN,NaN,1
101,NaN,INSIDER,International food,Accepting my yums,NaN,NaN,NaN,NaN,1
130,NaN,INSIDER,International food,NaN,NaN,NaN,NaN,NaN,1
157,NaN,INSIDER,European,NaN,NaN,NaN,NaN,NaN,1
174,NaN,INSIDER,Portuguese,NaN,NaN,NaN,NaN,NaN,1
246,NaN,INSIDER,Portuguese,NaN,NaN,NaN,NaN,NaN,1
249,NaN,INSIDER,European,NaN,NaN,NaN,NaN,NaN,1
262,NaN,INSIDER,Italian,NaN,NaN,NaN,NaN,NaN,1
269,NaN,Portuguese,NaN,NaN,NaN,NaN,NaN,NaN,1
286,NaN,INSIDER,European,NaN,NaN,NaN,NaN,NaN,1


In [116]:
df_tags = df_tags.replace(['YUMS x2'], np.nan)

In [117]:
df_tags = df_tags.replace(['Accepting my yums'], np.nan)

In [118]:
df_tags = df_tags.replace(['INSIDER'], np.nan)

In [119]:
df_tags.head(20)

,tags/0,tags/1,tags/2,tags/3,tags/4,tags/5,tags/6,tags/7,MICHELIN
0,NaN,International food,NaN,NaN,NaN,NaN,NaN,NaN,0
1,International food,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,Japanese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,Indian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,Portuguese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,NaN,Italian,NaN,NaN,NaN,NaN,NaN,NaN,0
6,Pizzeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,NaN,International food,NaN,NaN,NaN,NaN,NaN,NaN,0
8,Portuguese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,NaN,NaN,Portuguese,NaN,NaN,NaN,NaN,NaN,1


In [120]:
#unique values in the df_tags dataframe
for col in df_tags.columns:
    print(df_tags[col].unique())

[nan 'International food' 'Japanese' 'Indian' 'Portuguese' 'Pizzeria'
 'International' 'Mediterranean' 'Fusion' 'Nepalese' 'Italian' 'European'
 'Seafood' 'Vegan cuisine' 'Traditional cuisine' 'Steakhouse' 'Regional'
 'Greek' 'Vegetarian' 'Varied' 'Grilled' 'Thai' 'Mexican' 'Local' 'French'
 'Pub grub' 'Brazilian' 'Venezuelan' 'Peruvian' 'Korean' 'American'
 'Meat Cuisine' 'Asian' 'Toscano' 'African' 'Syrian' 'Iranian' 'Lebanese'
 'Tibetan' 'Vietnamese' 'Argentinian' 'Pugliese' 'Spanish']
['International food' nan 'Italian' 'Japanese' 'Portuguese' 'European'
 'Traditional cuisine' 'Welcome' 'Varied' 'International' 'Asian'
 'Mediterranean' 'Thai' 'Steakhouse' 'South American' 'Fusion' 'Pizzeria'
 'Meat Cuisine' 'Brazilian' 'Seafood' 'Regional' 'Grilled' 'Mexican'
 'Spanish' 'Indian' 'Peruvian' 'Cantonese' 'Vegan cuisine' 'Local'
 'French' 'Chinese']
[nan 'Portuguese' 'International food' 'Indian' 'European' 'Italian'
 'Mediterranean' 'Japanese' 'International' 'Vegetarian'
 'Traditiona

In [121]:
# remove the last 5 columns from the dataframe
df_tags.drop(df_tags.columns[-6:], axis=1, inplace=True)

In [122]:
for col in df_tags.columns:
    print(df_tags[col].unique())

[nan 'International food' 'Japanese' 'Indian' 'Portuguese' 'Pizzeria'
 'International' 'Mediterranean' 'Fusion' 'Nepalese' 'Italian' 'European'
 'Seafood' 'Vegan cuisine' 'Traditional cuisine' 'Steakhouse' 'Regional'
 'Greek' 'Vegetarian' 'Varied' 'Grilled' 'Thai' 'Mexican' 'Local' 'French'
 'Pub grub' 'Brazilian' 'Venezuelan' 'Peruvian' 'Korean' 'American'
 'Meat Cuisine' 'Asian' 'Toscano' 'African' 'Syrian' 'Iranian' 'Lebanese'
 'Tibetan' 'Vietnamese' 'Argentinian' 'Pugliese' 'Spanish']
['International food' nan 'Italian' 'Japanese' 'Portuguese' 'European'
 'Traditional cuisine' 'Welcome' 'Varied' 'International' 'Asian'
 'Mediterranean' 'Thai' 'Steakhouse' 'South American' 'Fusion' 'Pizzeria'
 'Meat Cuisine' 'Brazilian' 'Seafood' 'Regional' 'Grilled' 'Mexican'
 'Spanish' 'Indian' 'Peruvian' 'Cantonese' 'Vegan cuisine' 'Local'
 'French' 'Chinese']
[nan 'Portuguese' 'International food' 'Indian' 'European' 'Italian'
 'Mediterranean' 'Japanese' 'International' 'Vegetarian'
 'Traditiona

In [123]:
# join these columns in one column just with the non null values
df_tags['type'] = df_tags.apply(lambda row: next((val for val in row if not pd.isna(val)), np.nan), axis=1)


In [125]:
# delete all the columns except the new one
df_tags.drop(df_tags.columns[:-1], axis=1, inplace=True)

In [126]:
# CREATE A CSV FILE WITH DF_TAGS DATAFRAME WITH NAME TYPES
df_tags.to_csv('types.csv', index = True)

#### PROMOS

In [ ]:
import random
# poderiamos eliminar as promoções do the fork e criar as nossas próprias promoções


# Define the days of the week
days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Define the promotion types
promotion_types = ['None', 'Happy Hour', '10% off', '20% off','30% off', 'Free dessert']

# Define the number of promotions and restaurants
num_promotions = 10 # o numero de promoções que quisermos
num_restaurants = len(df_workable['name']) # o nomero total de restaurantes

# Create a list of promotion schedules
promotion_schedules = []

# Loop over each promotion
for i in range(num_promotions):
    # Choose a random day of the week
    day_of_week = random.choice(days_of_week)
    # Choose a random promotion type
    promotion_type = random.choice(promotion_types)

    # Choose a random restaurant ID
    restaurant_id = random.randint(1, num_restaurants)

    # Choose a random start time and end time based on promotion type
    if promotion_type == 'Happy Hour':
        start_time = f"{random.randint(2, 4)}:{random.choice(['00', '15', '30', '45'])}pm"
        end_time = f"{random.randint(5, 7)}:{random.choice(['00', '15', '30', '45'])}pm"
    elif promotion_type == '20% off':
        start_time = f"{random.randint(6, 8)}:{random.choice(['00', '15', '30', '45'])}pm"
        end_time = f"{random.randint(9, 11)}:{random.choice(['00', '15', '30', '45'])}pm"
    else:
        start_time = f"{random.randint(5, 7)}:{random.choice(['00', '15', '30', '45'])}pm"
        end_time = f"{random.randint(8, 10)}:{random.choice(['00', '15', '30', '45'])}pm"

    # Add the promotion schedule to the list
    promotion_schedules.append({
        'day_of_week': day_of_week,
        'start_time': start_time,
        'end_time': end_time,
        'promotion_type': promotion_type,
        'restaurant_id': restaurant_id
    })

# Print the promotion schedules
for i, schedule in enumerate(promotion_schedules):
    print(f"Promotion {i+1}: {schedule['promotion_type']} at Restaurant {schedule['restaurant_id']} on {schedule['day_of_week']} from {schedule['start_time']} to {schedule['end_time']}")


#### CHEFS

In [130]:
df_workable['chefName'].value_counts()

chefName
Not Applicable               1000
Carlos Guerreiro                6
Rui Sequeira                    4
Luís Candeias                   4
Julio Ferradas                  4
                             ... 
D. Miguel Sebastião             1
Mário Lino e Marco Morais       1
Hugo Teixeira                   1
Nuno Martins                    1
Miguel Frias                    1
Name: count, Length: 542, dtype: int64

In [131]:
chefs = df_workable[['chefName']]

# Save this DataFrame to a CSV file
chefs= chefs.sort_values(by='chefName')
chefs.to_csv('chefs.csv', index=False)

In [132]:
import re
chefs['chefName'] = chefs['chefName'].str.replace(r'[^A-Za-z\s]', '', regex=True)
chefs['chefName'] = chefs['chefName'].str.replace('Chefes', '', case=False).str.strip()
chefs['chefName'] = chefs['chefName'].str.replace('Chefe', '', case=False).str.strip()
chefs['chefName'] = chefs['chefName'].str.replace('Chef', '', case=False).str.strip()
chefs['chefName'] = chefs['chefName'].str.replace('executivos', '', case=False).str.strip()
chefs['chefName'] = chefs['chefName'].str.replace('executivo', '', case=False).str.strip()
chefs['chefName'] = chefs['chefName'].str.replace('Chefe executivo', '', case=False).str.strip()
chefs['chefName'] = chefs['chefName'].str.replace('Chef', '', case=False).str.strip()
chefs['chefName'] = chefs['chefName'].str.replace('  ', ' ', case=False).str.strip()
chefs.to_csv('chefs.csv', index=False)


#### General Metrics

In [48]:
df_workable = data[['address', 'averagePrice', 'chefName', 'cuisine', 'currency', 'description', 'hasLoyaltyProgram', 'isBookable', 'maxPartySize',
 'name', 'offer',  'phone', 'photo', 'ratingValue', 'reviewCount', 'style', 'latitude', 'longitude', 'location', 'city',
 'ambienceRatingSummary', 'foodRatingSummary', 'serviceRatingSummary', 'paymentAcceptedSummary', ]].copy()

In [ ]:
# #If not euros, convert the average price to euros.
# c = CurrencyConverter()
# df_workable['averagePriceEUR'] = df_workable.apply(to_euros, axis=1)

In [49]:
df_workable.isnull().any()

address                   False
averagePrice              False
chefName                  False
cuisine                   False
currency                  False
description                True
hasLoyaltyProgram         False
isBookable                False
maxPartySize               True
name                      False
offer                     False
phone                     False
photo                     False
ratingValue                True
reviewCount                True
style                      True
latitude                  False
longitude                 False
location                  False
city                      False
ambienceRatingSummary      True
foodRatingSummary          True
serviceRatingSummary       True
paymentAcceptedSummary    False
dtype: bool

### Visualizing Data

In [50]:
df_workable['style'].unique()

array(['Bachelor party', 'After work', 'Birthday', 'Good for ceremonies',
       'All you can eat buffet', 'Good for families',
       'Contemporary cuisine', 'Author', 'Good for groups',
       'Central location', 'Bistro', nan, 'Fine Dining', 'Cosy',
       'Live music', 'Creative', 'Brunch', 'Lunch', 'Traditional',
       'Bistronomic', 'Good for a business lunch', 'Terrace',
       'Homemade cuisine', 'Brasserie', 'Garden', 'Oceanfront',
       'Great view', 'Café', 'Trendy', 'Romantic', 'Restaurant hotel',
       'Communion', 'Breakfast', 'Winter terrace', 'Nightlife',
       'With friends', 'Street Food', 'Organic', 'Healthy.old', 'Ethnic',
       'From market', 'Design', 'Wine bar'], dtype=object)

In [ ]:
import folium

m = folium.Map(location=(38.736946, -9.142685), zoom_start=7, tiles="cartodb positron")


In [ ]:
from folium.plugins import HeatMap
HeatMap(data=df_workable[['latitude', 'longitude', 'averagePriceEUR']], 
            radius=3, max_zoom=13, ).add_to(m)

m
#gradient={0.2: 'gray', 0.4: 'green', 0.6: 'pink', 0.8: 'orange', 1.0: 'red'}